In [ ]:
import numpy as np
import pandas as pd


df = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv", dtype={"article_id": str})
print(df.shape)
df.head()

In [ ]:
af = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/articles.csv", dtype={"article_id": str,'product_code':str,'product_type_no':str,'graphical_appearance_no':str,'colour_group_code':str,'perceived_colour_value_id':str,'perceived_colour_master_id':str,'department_no':str,'index_code':str,'index_group_no':str,'section_no':str,'garment_group_no':str}) #convert

In [ ]:
af.drop(labels=['prod_name','product_type_name','product_group_name','graphical_appearance_name','colour_group_name','perceived_colour_value_name','perceived_colour_master_name','department_name','index_name','index_group_name','index_name','index_group_name','section_name','garment_group_name','detail_desc'], axis=1,inplace=True)

In [ ]:
af.head()

In [ ]:
df["t_dat"] = pd.to_datetime(df["t_dat"])
df["t_dat"].max()

In [ ]:
active_articles = df.groupby("article_id")["t_dat"].max().reset_index()
active_articles = active_articles[active_articles["t_dat"] >= "2019-09-01"].reset_index()
active_articles.shape

In [ ]:
df = df[df["article_id"].isin(active_articles["article_id"])].reset_index(drop=True)
df.shape

In [ ]:
af = af[af["article_id"].isin(active_articles["article_id"])].reset_index(drop=True)
af.shape

In [ ]:
df["week"] = (df["t_dat"].max() - df["t_dat"]).dt.days // 7
df["week"].value_counts()

In [ ]:
new_row = pd.DataFrame({'article_id':'#', 'product_code':'#', 'product_type_no':'#',
       'graphical_appearance_no':'#','colour_group_code':'#',
       'perceived_colour_value_id':'#','perceived_colour_master_id':'#',
       'department_no':'#','index_code':'#','index_group_no':'#', 'section_no':'#',
       'garment_group_no':'#'}, index =[0])###placeholder

In [ ]:
af = pd.concat([new_row,af[:]]).reset_index(drop=True)#add placeholders

In [ ]:
af.head()

In [ ]:
af.sort_values(by='article_id',inplace = True)#make sure each index represent each article_id

In [ ]:
from sklearn.preprocessing import LabelEncoder

article_ids = np.unique(af["article_id"].values)

le_article = LabelEncoder()
le_article.fit(article_ids)

af["article_id"] = le_article.transform(af["article_id"])
df["article_id"] = le_article.transform(df["article_id"])

In [ ]:
#incorporate other features
f1 = np.unique(af['product_code'].values)
f2 = np.unique(af['product_type_no'].values)
f3 = np.unique(af['graphical_appearance_no'].values)
f4 = np.unique(af['colour_group_code'].values)
f5 = np.unique(af['perceived_colour_value_id'].values)
f6 = np.unique(af['perceived_colour_master_id'].values)
f7 = np.unique(af['department_no'].values)
f8 = np.unique(af['index_code'].values)
f9 = np.unique(af['index_group_no'].values)
f10 = np.unique(af['section_no'].values)
f11 = np.unique(af['garment_group_no'].values)

In [ ]:
le_f1 = LabelEncoder()
le_f2 = LabelEncoder()
le_f3 = LabelEncoder()
le_f4 = LabelEncoder()
le_f5 = LabelEncoder()
le_f6 = LabelEncoder()
le_f7 = LabelEncoder()
le_f8 = LabelEncoder()
le_f9 = LabelEncoder()
le_f10 = LabelEncoder()
le_f11 = LabelEncoder()
le_f1.fit(f1)
le_f2.fit(f2)
le_f3.fit(f3)
le_f4.fit(f4)
le_f5.fit(f5)
le_f6.fit(f6)
le_f7.fit(f7)
le_f8.fit(f8)
le_f9.fit(f9)
le_f10.fit(f10)
le_f11.fit(f11)
af['product_code'] = le_f1.transform(af['product_code'])
af['product_type_no'] = le_f2.transform(af['product_type_no'])
af['graphical_appearance_no'] = le_f3.transform(af['graphical_appearance_no'])
af['colour_group_code'] = le_f4.transform(af['colour_group_code'])
af['perceived_colour_value_id'] = le_f5.transform(af['perceived_colour_value_id'])
af['perceived_colour_master_id'] = le_f6.transform(af['perceived_colour_master_id'])
af['department_no'] = le_f7.transform(af['department_no'])
af['index_code'] = le_f8.transform(af['index_code'])
af['index_group_no'] = le_f9.transform(af['index_group_no'])
af['section_no'] = le_f10.transform(af['section_no'])
af['garment_group_no'] = le_f11.transform(af['garment_group_no'])

In [ ]:
af.head()

In [ ]:
WEEK_HIST_MAX = 5

def create_dataset(df, week):
    hist_df = df[(df["week"] > week) & (df["week"] <= week + WEEK_HIST_MAX)]
    hist_df = hist_df.groupby("customer_id").agg({"article_id": list, "week": list}).reset_index()
    hist_df.rename(columns={"week": 'week_history'}, inplace=True)
    
    target_df = df[df["week"] == week]
    target_df = target_df.groupby("customer_id").agg({"article_id": list}).reset_index()
    target_df.rename(columns={"article_id": "target"}, inplace=True)
    target_df["week"] = week
    
    return target_df.merge(hist_df, on="customer_id", how="left")

val_weeks = [0]
train_weeks = [1, 2, 3, 4]


val_df = pd.concat([create_dataset(df, w) for w in val_weeks]).reset_index(drop=True)
train_df = pd.concat([create_dataset(df, w) for w in train_weeks]).reset_index(drop=True)
train_df.shape, val_df.shape

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch
from tqdm import tqdm

class HMDataset(Dataset):
    def __init__(self, df, seq_len, is_test=False):
        self.df = df.reset_index(drop=True)
        self.seq_len = seq_len
        self.is_test = is_test
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index):
        row = self.df.iloc[index]
        
        if self.is_test:
            target = torch.zeros(2).float()
        else:
            target = torch.zeros(len(article_ids)).float()
            for t in row.target:
                target[t] = 1.0
            
        article_hist = torch.zeros(self.seq_len).long()
        week_hist = torch.ones(self.seq_len).float()
        
        article_hist_np = np.zeros(self.seq_len).astype(int)
        
        
        if isinstance(row.article_id, list):
            if len(row.article_id) >= self.seq_len:
                article_hist = torch.LongTensor(row.article_id[-self.seq_len:])
                week_hist = (torch.LongTensor(row.week_history[-self.seq_len:]) - row.week)/WEEK_HIST_MAX/2
            else:
                article_hist[-len(row.article_id):] = torch.LongTensor(row.article_id)
                week_hist[-len(row.article_id):] = (torch.LongTensor(row.week_history) - row.week)/WEEK_HIST_MAX/2
        
        
        if isinstance(row.article_id, list):
            if len(row.article_id) >= self.seq_len:
                article_hist_np = np.array(row.article_id[-self.seq_len:])
            else:
                article_hist_np[-len(row.article_id):] = np.array(row.article_id)
                
        ##2 hours for inference        
        #print(article_hist_np)
        
        vfunc = np.vectorize(lambda x: af.iloc[x,1])
        f1 = torch.from_numpy(vfunc(article_hist_np))
        vfunc = np.vectorize(lambda x: af.iloc[x,2])
        f2 = torch.from_numpy(vfunc(article_hist_np))
        vfunc = np.vectorize(lambda x: af.iloc[x,3])
        f3 = torch.from_numpy(vfunc(article_hist_np))
        vfunc = np.vectorize(lambda x: af.iloc[x,4])
        f4 = torch.from_numpy(vfunc(article_hist_np))
        vfunc = np.vectorize(lambda x: af.iloc[x,5])
        f5 = torch.from_numpy(vfunc(article_hist_np))
        vfunc = np.vectorize(lambda x: af.iloc[x,6])
        f6 = torch.from_numpy(vfunc(article_hist_np))
        vfunc = np.vectorize(lambda x: af.iloc[x,7])
        f7 = torch.from_numpy(vfunc(article_hist_np))
        vfunc = np.vectorize(lambda x: af.iloc[x,8])
        f8 = torch.from_numpy(vfunc(article_hist_np))
        vfunc = np.vectorize(lambda x: af.iloc[x,9])
        f9 = torch.from_numpy(vfunc(article_hist_np))
        vfunc = np.vectorize(lambda x: af.iloc[x,10])
        f10 = torch.from_numpy(vfunc(article_hist_np))
        vfunc = np.vectorize(lambda x: af.iloc[x,11])
        f11 = torch.from_numpy(vfunc(article_hist_np))


#         2 hours for inference
#         article_hist1 = copy.deepcopy(article_hist)
#         f1 = article_hist1.apply_(lambda x : af.iloc[x,1])
#         article_hist2 = copy.deepcopy(article_hist)
#         f2 = article_hist2.apply_(lambda x : af.iloc[x,2])
#         article_hist3 = copy.deepcopy(article_hist)
#         f3 = article_hist3.apply_(lambda x : af.iloc[x,3])
#         article_hist4 = copy.deepcopy(article_hist)
#         f4 = article_hist4.apply_(lambda x : af.iloc[x,4])
#         article_hist5 = copy.deepcopy(article_hist)
#         f5 = article_hist5.apply_(lambda x : af.iloc[x,5])
#         article_hist6 = copy.deepcopy(article_hist)
#         f6 = article_hist6.apply_(lambda x : af.iloc[x,6])
#         article_hist7 = copy.deepcopy(article_hist)
#         f7 = article_hist7.apply_(lambda x : af.iloc[x,7])
#         article_hist8 = copy.deepcopy(article_hist)
#         f8 = article_hist8.apply_(lambda x : af.iloc[x,8])
#         article_hist9 = copy.deepcopy(article_hist)
#         f9 = article_hist9.apply_(lambda x : af.iloc[x,9])
#         article_hist10 = copy.deepcopy(article_hist)
#         f10 = article_hist10.apply_(lambda x : af.iloc[x,10])
#         article_hist11 = copy.deepcopy(article_hist)
#         f11 = article_hist11.apply_(lambda x : af.iloc[x,11])

#         4 hours for inference
#         article_hist_ = pd.DataFrame(article_hist.cpu().numpy())
#         f1 = torch.from_numpy(article_hist_.applymap(lambda x: af.iloc[x,1]).to_numpy()).squeeze()
#         f2 = torch.from_numpy(article_hist_.applymap(lambda x: af.iloc[x,2]).to_numpy()).squeeze()
#         f3 = torch.from_numpy(article_hist_.applymap(lambda x: af.iloc[x,3]).to_numpy()).squeeze()
#         f4 = torch.from_numpy(article_hist_.applymap(lambda x: af.iloc[x,4]).to_numpy()).squeeze()
#         f5 = torch.from_numpy(article_hist_.applymap(lambda x: af.iloc[x,5]).to_numpy()).squeeze()
#         f6 = torch.from_numpy(article_hist_.applymap(lambda x: af.iloc[x,6]).to_numpy()).squeeze()
#         f7 = torch.from_numpy(article_hist_.applymap(lambda x: af.iloc[x,7]).to_numpy()).squeeze()
#         f8 = torch.from_numpy(article_hist_.applymap(lambda x: af.iloc[x,8]).to_numpy()).squeeze()
#         f9 = torch.from_numpy(article_hist_.applymap(lambda x: af.iloc[x,9]).to_numpy()).squeeze()
#         f10 = torch.from_numpy(article_hist_.applymap(lambda x: af.iloc[x,10]).to_numpy()).squeeze()
#         f11 = torch.from_numpy(article_hist_.applymap(lambda x: af.iloc[x,11]).to_numpy()).squeeze()

        
        return article_hist, week_hist, target, (f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11)
    
HMDataset(val_df, 64)[1]

In [ ]:
all_f1 = torch.from_numpy(af.iloc[:,1].values).cuda()
all_f2 = torch.from_numpy(af.iloc[:,2].values).cuda()
all_f3 = torch.from_numpy(af.iloc[:,3].values).cuda()
all_f4 = torch.from_numpy(af.iloc[:,4].values).cuda()
all_f5 = torch.from_numpy(af.iloc[:,5].values).cuda()
all_f6 = torch.from_numpy(af.iloc[:,6].values).cuda()
all_f7 = torch.from_numpy(af.iloc[:,7].values).cuda()
all_f8 = torch.from_numpy(af.iloc[:,8].values).cuda()
all_f9 = torch.from_numpy(af.iloc[:,9].values).cuda()
all_f10 = torch.from_numpy(af.iloc[:,10].values).cuda()
all_f11 = torch.from_numpy(af.iloc[:,11].values).cuda()
all_f = (all_f1,all_f2,all_f3,all_f4,all_f5,all_f6,all_f7,all_f8,all_f9,all_f10,all_f11)

In [ ]:
all_f

In [ ]:
def adjust_lr(optimizer, epoch):
    if epoch < 1:
        lr = 5e-5
    elif epoch < 6:
        lr = 1e-3
    elif epoch < 9:
        lr = 1e-4
    else:
        lr = 1e-5

    for p in optimizer.param_groups:
        p['lr'] = lr
    return lr
    
def get_optimizer(net):
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, net.parameters()), lr=3e-4, betas=(0.9, 0.999),
                                 eps=1e-08)
    return optimizer

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class HMModel(nn.Module):
    def __init__(self, article_shape,f1_shape,f2_shape,f3_shape,f4_shape,f5_shape,f6_shape,f7_shape,f8_shape,f9_shape,f10_shape,f11_shape,all_f):
        super(HMModel, self).__init__()
        
        self.all_f1 = all_f[0]
        self.all_f2 = all_f[1]
        self.all_f3 = all_f[2]
        self.all_f4 = all_f[3]
        self.all_f5 = all_f[4]
        self.all_f6 = all_f[5]
        self.all_f7 = all_f[6]
        self.all_f8 = all_f[7]
        self.all_f9 = all_f[8]
        self.all_f10 = all_f[9]
        self.all_f11 = all_f[10]
        
        self.article_emb = nn.Embedding(article_shape[0], embedding_dim=article_shape[1],padding_idx=0)
        
        self.f1_emb = nn.Embedding(f1_shape[0], embedding_dim=f1_shape[1],padding_idx=0)
        self.f2_emb = nn.Embedding(f2_shape[0], embedding_dim=f2_shape[1],padding_idx=0)
        self.f3_emb = nn.Embedding(f3_shape[0], embedding_dim=f3_shape[1],padding_idx=0)
        self.f4_emb = nn.Embedding(f4_shape[0], embedding_dim=f4_shape[1],padding_idx=0)
        self.f5_emb = nn.Embedding(f5_shape[0], embedding_dim=f5_shape[1],padding_idx=0)
        self.f6_emb = nn.Embedding(f6_shape[0], embedding_dim=f6_shape[1],padding_idx=0)
        self.f7_emb = nn.Embedding(f7_shape[0], embedding_dim=f7_shape[1],padding_idx=0)
        self.f8_emb = nn.Embedding(f8_shape[0], embedding_dim=f8_shape[1],padding_idx=0)
        self.f9_emb = nn.Embedding(f9_shape[0], embedding_dim=f9_shape[1],padding_idx=0)
        self.f10_emb = nn.Embedding(f10_shape[0], embedding_dim=f10_shape[1],padding_idx=0)
        self.f11_emb = nn.Embedding(f11_shape[0], embedding_dim=f11_shape[1],padding_idx=0)
        
        self.article_likelihood = nn.Parameter(torch.zeros(article_shape[0]), requires_grad=True)
        
        self.top = nn.Sequential(nn.Conv1d(3, 8, kernel_size=1), nn.LeakyReLU(),
                                 nn.Conv1d(8, 3, kernel_size=1), nn.LeakyReLU(),
                                 nn.Conv1d(3, 1, kernel_size=1))
        
    def forward(self, inputs, f_):
        article_hist, week_hist = inputs[0], inputs[1]
        
        x = self.article_emb(article_hist)

        #Get the embedding for articles in article_hist.
        x1 = self.f1_emb(f_[0])
        x_m = torch.cat((x,x1),2)##[256,16,317]
        x2 = self.f2_emb(f_[1])
        x_m = torch.cat((x_m,x2),2)###[256,16,322]
        x3 = self.f3_emb(f_[2])
        x_m = torch.cat((x_m,x3),2)###[256,16,327]
        x4 = self.f4_emb(f_[3])
        x_m = torch.cat((x_m,x4),2)###[256,16,332]
        x5 = self.f5_emb(f_[4])
        x_m = torch.cat((x_m,x5),2)###[256,16,337]
        
        x6 = self.f6_emb(f_[5])
        x_m = torch.cat((x_m,x6),2)###[256,16,342]
        x7 = self.f7_emb(f_[6])
        x_m = torch.cat((x_m,x7),2)###[256,16,347]
        x8 = self.f8_emb(f_[7])
        x_m = torch.cat((x_m,x2),2)###[256,16,352]
        x9 = self.f9_emb(f_[8])
        x_m = torch.cat((x_m,x2),2)###[256,16,357]
        x10 = self.f10_emb(f_[9])
        x_m = torch.cat((x_m,x2),2)###[256,16,362]
        x11 = self.f11_emb(f_[10])
        x_m = torch.cat((x_m,x11),2)###[256,16,367]
        
        
        
        
        #Get the embedding for all articles
        x_all = self.article_emb.weight###[105543, 312]
        f1_all = self.f1_emb(self.all_f1)###[105543, 10]
        all_ = torch.cat((x_all,f1_all),1)###[105543, 317]
        f2_all = self.f2_emb(self.all_f2)
        all_ = torch.cat((all_,f2_all),1)###[105543, 322]
        f3_all = self.f3_emb(self.all_f3)
        all_ = torch.cat((all_,f3_all),1)###[105543, 327]
        f4_all = self.f4_emb(self.all_f4)
        all_ = torch.cat((all_,f4_all),1)###[105543, 332]
        f5_all = self.f5_emb(self.all_f5) 
        all_ = torch.cat((all_,f5_all),1)###[105543, 337]
        
        f6_all = self.f6_emb(self.all_f6)
        all_ = torch.cat((all_,f6_all),1)###[105543, 342]
        f7_all = self.f7_emb(self.all_f7)
        all_ = torch.cat((all_,f7_all),1)###[105543, 347]
        f8_all = self.f8_emb(self.all_f8)
        all_ = torch.cat((all_,f8_all),1)###[105543, 352]
        f9_all = self.f9_emb(self.all_f9)
        all_ = torch.cat((all_,f9_all),1)###[105543, 357]
        f10_all = self.f10_emb(self.all_f10)
        all_ = torch.cat((all_,f10_all),1)###[105543, 362]
        f11_all = self.f11_emb(self.all_f11)
        all_ = torch.cat((all_,f11_all),1)###[105543, 367]
    
        x_m = F.normalize(x_m, dim=2)
        
        x = x_m@F.normalize(all_).T ###[256,16,105543]
        
        x, indices = x.max(axis=1)
        x = x.clamp(1e-3, 0.999)
        x = -torch.log(1/x - 1)
        
        max_week = week_hist.unsqueeze(2).repeat(1, 1, x.shape[-1]).gather(1, indices.unsqueeze(1).repeat(1, week_hist.shape[1], 1))
        max_week = max_week.mean(axis=1).unsqueeze(1)
        
        x = torch.cat([x.unsqueeze(1), max_week,
                       self.article_likelihood[None, None, :].repeat(x.shape[0], 1, 1)], axis=1)
        
        x = self.top(x).squeeze(1)
         
        return x

model = HMModel((len(le_article.classes_), 312),(len(le_f1.classes_),10),(len(le_f2.classes_),10),(len(le_f3.classes_),10),(len(le_f4.classes_),10),(len(le_f5.classes_),10),(len(le_f6.classes_),10),(len(le_f7.classes_),10),(len(le_f8.classes_),10),(len(le_f9.classes_),10),(len(le_f10.classes_),10),(len(le_f11.classes_),10),all_f)
model = model.cuda()

In [ ]:
import sys

def calc_map(topk_preds, target_array, k=12):
    metric = []
    tp, fp = 0, 0
    
    for pred in topk_preds:
        if target_array[pred]:
            tp += 1
            metric.append(tp/(tp + fp))
        else:
            fp += 1
            
    return np.sum(metric) / min(k, target_array.sum())

def read_data(data):

    return tuple(d.cuda() for d in data[:2]),tuple(d.cuda() for d in data[3]), data[2].cuda()


def validate(model, val_loader, k=12):
    model.eval()
    
    tbar = tqdm(val_loader, file=sys.stdout)
    
    maps = []
    
    with torch.no_grad():
        for idx, data in enumerate(tbar):
            inputs, f, target = read_data(data)


            logits = model(inputs,f)

            _, indices = torch.topk(logits, k, dim=1)

            indices = indices.detach().cpu().numpy()
            target = target.detach().cpu().numpy()

            for i in range(indices.shape[0]):
                maps.append(calc_map(indices[i], target[i]))
        
    
    return np.mean(maps)

SEQ_LEN = 16

BS = 256
NW = 8

val_dataset = HMDataset(val_df, SEQ_LEN)

val_loader = DataLoader(val_dataset, batch_size=BS, shuffle=False, num_workers=NW,
                          pin_memory=False, drop_last=False)


### Train and validate

In [ ]:
def dice_loss(y_pred, y_true):
    y_pred = y_pred.sigmoid()
    intersect = (y_true*y_pred).sum(axis=1)
    
    return 1 - (intersect/(intersect + y_true.sum(axis=1) + y_pred.sum(axis=1))).mean()


def train(model, train_loader, val_loader, epochs):
    np.random.seed(SEED)
    
    optimizer = get_optimizer(model)
    scaler = torch.cuda.amp.GradScaler()

    criterion = torch.nn.BCEWithLogitsLoss()
    
    for e in range(epochs):
        model.train()
        tbar = tqdm(train_loader, file=sys.stdout)
        
        lr = adjust_lr(optimizer, e)
        
        loss_list = []

        for idx, data in enumerate(tbar):
            inputs, f, target = read_data(data)
            
            optimizer.zero_grad()
            
            with torch.cuda.amp.autocast():
                logits = model(inputs,f)

                loss = criterion(logits, target) + dice_loss(logits, target)
            
            
            #loss.backward()
            scaler.scale(loss).backward()
            #optimizer.step()
            scaler.step(optimizer)
            scaler.update()
            
            loss_list.append(loss.detach().cpu().item())
            
            avg_loss = np.round(100*np.mean(loss_list), 4)

            tbar.set_description(f"Epoch {e+1} Loss: {avg_loss} lr: {lr}")
            
            
        #val_map = validate(model, val_loader)

        #log_text = f"Epoch {e+1}\nTrain Loss: {avg_loss}\nValidation MAP: {val_map}\n"
        
            
        #print(log_text)
        
        #logfile = open(f"models/{MODEL_NAME}_{SEED}.txt", 'a')
        #logfile.write(log_text)
        #logfile.close()
    return model


MODEL_NAME = "exp001"
SEED = 0

train_dataset = HMDataset(train_df, SEQ_LEN)

train_loader = DataLoader(train_dataset, batch_size=BS, shuffle=True, num_workers=NW,
                          pin_memory=False, drop_last=True)

model = train(model, train_loader, val_loader, epochs=5)

In [ ]:
len(read_data(train_dataset[0])[1])

### Finetune with more recent data for submission (include validation set)

In [ ]:
train_dataset = HMDataset(train_df[train_df["week"] < 4].append(val_df), SEQ_LEN)
train_loader = DataLoader(train_dataset, batch_size=BS, shuffle=True, num_workers=NW,
                          pin_memory=False, drop_last=True)

model = train(model, train_loader, val_loader, epochs=2)

In [ ]:
test_df = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv').drop("prediction", axis=1)
print(test_df.shape)
test_df.head()

In [ ]:
def create_test_dataset(test_df):
    week = -1
    test_df["week"] = week
    
    hist_df = df[(df["week"] > week) & (df["week"] <= week + WEEK_HIST_MAX)]
    hist_df = hist_df.groupby("customer_id").agg({"article_id": list, "week": list}).reset_index()
    hist_df.rename(columns={"week": 'week_history'}, inplace=True)
    
    
    return test_df.merge(hist_df, on="customer_id", how="left")

test_df = create_test_dataset(test_df)
test_df.head()

In [ ]:
test_df["article_id"].isnull().mean()

In [ ]:
test_ds = HMDataset(test_df, SEQ_LEN, is_test=True)
test_loader = DataLoader(test_ds, batch_size=BS, shuffle=False, num_workers=NW,
                          pin_memory=False, drop_last=False)


def inference(model, loader, k=12):
    model.eval()
    
    tbar = tqdm(loader, file=sys.stdout)
    
    preds = []
    
    with torch.no_grad():
        for idx, data in enumerate(tbar):
            inputs,f, target = read_data(data)

            logits = model(inputs,f)

            _, indices = torch.topk(logits, k, dim=1)

            indices = indices.detach().cpu().numpy()
            target = target.detach().cpu().numpy()

            for i in range(indices.shape[0]):
                preds.append(" ".join(list(le_article.inverse_transform(indices[i]))))
        
    
    return preds


test_df["prediction"] = inference(model, test_loader)

In [ ]:
test_df.to_csv("submission.csv", index=False, columns=["customer_id", "prediction"])